In [ ]:
# using Distributed
# nprocs() < 2 && addprocs(4);

In [ ]:
using Plots
default(fmt = :png)
using LinearAlgebra
# using Revise
using LatticeQM

# Tutorial: Tight-binding operators

In [ ]:
lat = Geometries.honeycomb_AB()
lat = Lattices.superlattice(lat, [[1,1] [2,-1]])
Lattices.foldPC!(lat; shift=[1/3,0,0])#; shift=[1/3,1/3,0])

# H = Operators.graphene_rhombohedral(lat; γ0=-1.0, γ1=0.3, γ2=-0.045, γ3=0.07, γ4=0.045)
H = Operators.graphene_rhombohedral(lat; γ0=-3.16)
Operators.addinterlayerbias!(H, lat, 0.090)
# Operators.setfilling!(H, 0.5003; nk=400, T=0)

posZ = Operators.positionalong(lat, [0,0,1.0]; rescale=true)
valley = Operators.valley(lat; spinhalf=false)

nothing

In [ ]:
a1 = Lattices.getA(lat)[:,1]
a2 = Lattices.getA(lat)[:,1]

plot(lat, 3; supercell=[0,0])#, filter=(3, z->z<0))

# plot!([0 a1[1]], [0,a1[2]], marker =:circle, arrow=(:closed, 2.0))

In [ ]:
ks = kpath(lat; num_points=2000)
bands = getbands(H, ks, [valley, posZ])

Spectrum.getberry!(bands, H, ks) # add berry curvature to band data

plot(bands, 2; markersize=6, ylims=(-0.2,0.2), size=(1400,200),  colorbar=true)#, colorbar_title="spin")
hline!([-0.115, -0.13, -0.14, -0.17])

In [ ]:
include("sampling.jl")
kmesh = getcustomkmesh_gamma(; N = 10, k = 2, l = 10)

scatter(kmesh.points[1, :], kmesh.points[2, :], aspect_ratio = :equal)

In [ ]:
include("sampling.jl")
kmesh = getcustomkmesh_gamma(; N = 8, k = 1, l = 150)

energies = LinRange(-0.1, 0.1, 500)
# kgrid = Structure.regulargrid(; nk = 400^2)

μs = Spectrum.chemicalpotential(H, kmesh, [0.475, 0.49, 0.495])

dos = Spectrum.getdos(H, energies, kmesh; format = :dense, Γ = 0.001)
plot(energies, dos; ylabel = "DOS", xlabel = "Energy [t]")
vline!(μs)

In [ ]:
bands, obs = Spectrum.bandmatrix(H, kmesh, [valley, posZ]; multimode = :distributed)
nothing

In [ ]:
density = Spectrum.fermisurfacedensity_fromdata(bands, μs; broadening = 0.001)
density_layer = Spectrum.fermisurfacedensity_fromdata(bands, μs, obs[:, :, 2]; broadening = 0.001)#; broadening=0.001)
density_valley = Spectrum.fermisurfacedensity_fromdata(bands, μs, obs[:, :, 1]; broadening = 0.001)#; broadening=0.001)

# using DelimitedFiles
# writedlm("output/density.out", density); writedlm("output/density_layer.out", density_layer)

nothing
#clims=(-1,1), color=:seismic
# scatter(kpoints[1,:], kpoints[2,:]; zcolor=vec(density[1,:]), markersize=1, markerstrokewidth=0, aspect_ratio=:equal)

In [ ]:
kgrid = inv(Lattices.getB(lat)[1:2, 1:2]) * kmesh.points

scatter(kgrid[1, :], kgrid[2, :]; zcolor = vec(density[3, :]), xlims = (-0.3, 0.3), ylims = (-0.3, 0.3), markersize=1.5, markerstrokewidth = 0, aspect_ratio = :equal)

In [ ]:
scatter(kgrid[1, :], kgrid[2, :]; zcolor = vec(density_valley[1, :]), clims = (-1, 1), xlims = (-0.3, 0.3), ylims = (-0.3, 0.3), color = :seismic, markersize = 2, markerstrokewidth = 0, aspect_ratio = :equal)

In [ ]:
using ProgressMeter;
ProgressMeter.ijulia_behavior(:clear);

μ = μs[1] #-0.14 #-0.105 # energies[3]

filling = Spectrum.filling(H, kmesh, μ)
Operators.setfilling!(H, kmesh, filling)

# sz = Operators.spin(lat, "sz")
valley = Operators.valley(lat; spinhalf = false)
posZ = Operators.positionalong(lat, [0, 0, 1.0]; rescale = true)
v = Operators.getcappedyukawa(lat; format = :dense, cellrange = 10, spin = false, a = 8.0, U = 4.5) # interaction potential

# valley = Operators.valley(lat, x->x[5]; spinhalf=true)
# posZ = kron(Operators.positionalong(lat, [0,0,1.0]; rescale=true), Diagonal(ones(2)))
# v = Operators.getcappedyukawa(lat; format = :dense, cellrange = 10, spin = true, a = 8.0, U = 4.5) # interaction potential
# v = Operators.gettanh(lat; format=:dense, spin=true, a=3.5, U=3.5)
ρ_init = Meanfield.initialguess(v, :random, :nonlocal; lat = lat) # initial guess
# ρ_init = Meanfield.initialguess(v, :ferro; lat=lat) # initial guess
nothing

In [ ]:
ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, filling, kmesh; iterations=350, tol=1e-5,# p_norm=Inf,
    T=0.002, β=0.25,  show_trace=true
)

nothing

# Mean-field (no pairing allowed)

In [ ]:
using ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [ ]:
lat = Geometries.honeycomb_AB()
lat = Lattices.superlattice(lat, [[1,1] [2,-1]])
Lattices.foldPC!(lat)#; shift=[1/3,-1/3,0])

H = Operators.graphene(lat; tz=0.3, ℓinter=0.08, ℓintra=0.04, cellrange=3, format=:dense, mode=:spinhalf)
H = DenseHops(H)
# Materials.addhaldane!(hops, lat, 0.1; spinhalf=true, mode=:anti)
Operators.addinterlayerbias!(H, lat, 0.3)

μ = energies[6] #-0.14 #-0.105 # energies[3]

filling = Spectrum.filling(H, μ; nk=100)
Operators.setfilling!(H, filling; nk=100^2)

sz = Operators.spin(lat, "sz")
valley = Operators.valley(lat; spinhalf=true)
# valley = Operators.valley(lat, x->x[5]; spinhalf=true)
posZ = kron(Operators.positionalong(lat, [0,0,1.0]; rescale=true), Diagonal(ones(2)))

v = Operators.getcappedyukawa(lat; format=:dense, cellrange=3, spin=true, a=4.0, U=1.4) # interaction potential
# v = Operators.gettanh(lat; format=:dense, spin=true, a=3.5, U=3.5)
ρ_init = Meanfield.initialguess(v, :random, :nonlocal; lat=lat) # initial guess
# ρ_init = Meanfield.initialguess(v, :ferro; lat=lat) # initial guess
nothing

In [ ]:
ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, filling; klin=50, iterations=350, tol=1e-5,# p_norm=Inf,
    T=0.002, β=0.25,  show_trace=true
)

nothing

In [ ]:
ks = kpath(lat; num_points=1000)
bands_mf = getbands(HMF.h, ks, [sz, valley, posZ]; multimode=:distributed2)
bands_mf.bands .-= HMF.μ # shift chemical potential to zero

nothing

In [ ]:
plot(bands_mf, 2; ylims=(-0.95,0.95), size=(1400,200), sortcolor=true,  colorbar=true)
# plot(bands_mf, 1)

In [ ]:
B = Lattices.getB(lat)[1:2,1:2]
kpoints = 0.1*(Structure.regulargrid(; nk=300^2) .- [0.5,0.5])
kgrid = inv(B)*kpoints

# Get band data
bands, obs = Spectrum.bandmatrix(HMF.h, kgrid, [posZ, valley, sz]; multimode=:distributed2)

using DelimitedFiles
mkpath("output_fs_mf")
writedlm("output_fs_mf/kpoints.out", kpoints)
writedlm("output_fs_mf/bandmatrix.out", bands)
writedlm("output_fs_mf/obsmatrix.out", obs)

# Compute Fermi surfaces
energies = [HMF.μ] #LinRange(-0.09,-0.16,20)
density = Spectrum.fermisurfacedensity_fromdata(bands, energies; broadening=0.0005)
density_layer = Spectrum.fermisurfacedensity_fromdata(bands, energies, obs[:,:,1]; broadening=0.0005)#; broadening=0.001)
density_valley = Spectrum.fermisurfacedensity_fromdata(bands, energies, obs[:,:,2]; broadening=0.0005)#; broadening=0.001)
density_spin = Spectrum.fermisurfacedensity_fromdata(bands, energies, obs[:,:,3]; broadening=0.0005)#; broadening=0.001)

writedlm("output_fs_mf/density.out", density)
writedlm("output_fs_mf/density_layer.out", density_layer)
writedlm("output_fs_mf/density_valley.out", density_valley)
writedlm("output_fs_mf/density_spin.out", density_spin)

In [ ]:
bands = getbands(H, ks, [sz, valley, posZ]; multimode=:distributed2)

plot(bands, 2; ylims=(-0.95,0.95), size=(1400,200),  colorbar=true)
# plot(bands, 3)

In [ ]:
# bands_mf = getbands(HMF.h, ks, [sz, valley, posZ])
# bands_mf.bands .-= HMF.μ # shift chemical potential to zero

# # plot(bands_mf, 1; ylims=(-0.35,0.35), size=(800,200), colorbar=true)#, colorbar_title="spin")
# p1 = plot(bands, 3; ylims=(-1.35,1.35), size=(800,200), colorbar=true)#, colorbar_title="spin")
# p2 = plot(bands_mf, 3; ylims=(-1.35,1.35), size=(800,200), colorbar=true)#, colorbar_title="spin")

# plot(p1,p2)

# Mean-field with pairing allowed

In [ ]:
lat = Geometries.honeycomb_ABC()
H = Operators.graphene(lat; tz=0.25, ℓinter=0.15, ℓintra=0.07, format=:dense, mode=:spinhalf)
H = BdGOperator(H)
# Materials.addhaldane!(hops, lat, 0.1; spinhalf=true, mode=:anti)
Operators.addinterlayerbias!(H, lat, 0.3)

v = Operators.getcappedyukawa(lat; format=:dense, spin=true, k0=0.7, U=1.6) # interaction potential

ρ0_init = initialguess(v, :random, :nonlocal)
Δ0_init = initialguess(v, :random, :nonlocal)
ρ_init = BdGOperator(ρ0_init, Δ0_init) # initial guess

ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, 0.5; klin=50, iterations=800, tol=1e-5,# p_norm=Inf,
    T=0.01, β=0.7,  show_trace=true, clear_trace=true
)
Operators.addchemicalpotential!(HMF.h, -HMF.μ)

nothing

In [ ]:
eOP = Superconductivity.electron(HMF.h)
bands_mf = getbands(HMF.h, ks, eOP.h)

# plot(bands_mf, 1; ylims=(-0.35,0.35), size=(800,200), colorbar=true)#, colorbar_title="spin")
p1 = plot(bands, 1; ylims=(-0.35,0.35), size=(800,200), colorbar=true)#, colorbar_title="spin")
p2 = plot(bands_mf, 1; ylims=(-0.35,0.35), size=(800,200), colorbar=true)#, colorbar_title="spin")

plot(p1,p2)

# Magnetic field, Hofstadter butterfly

In [ ]:
lat = Geometries.honeycomb_ABC()
H = Operators.graphene(lat; tz=0.45, ℓinter=0.01, ℓintra=0.01, format=:dense, mode=:nospin)
# Materials.addhaldane!(hops, lat, 0.1; spinhalf=true, mode=:anti)
# Operators.addinterlayerbias!(H, lat, 0.3)

fluxes, energies = Operators.hofstadter(H, lat, 50);
nothing

In [ ]:
p = plot()
for (ϕ,Es)=zip(fluxes,energies)
    scatter!(p, repeat([ϕ],length(Es)), Es; markersize=0.75, markercolor=:black, legend=false)
end
plot!(p, xlabel="flux per unit cell", ylabel="Energy E/t")
plot!(p, size=(400,400))
p